<a href="https://colab.research.google.com/github/IYashCanCode/Bank-Churn/blob/main/Data_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><b>Feature Engineering

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
class Feature_Engineering:

  def Binary_encoding(self,dataframe,change_columns=None):                                          #Binary encoding : To encode any number or all of features that have 2 categories in them
    if change_columns is None:
      self.binary_columns = [ i for i in dataframe.columns if dataframe[i].nunique()==2]            #Encodes all the columns having 2 categories in them
    else:
      self.binary_columns = [ i for i in change_columns if dataframe[i].nunique()==2]               #Encodes only those columns having 2 categories in them which you have passed in the parameter change_columns(note: column(s) must be passed as list of column(s) )


    for i in self.binary_columns:
      dataframe[i].replace({dataframe[i].unique()[0]:0,dataframe[i].unique()[1]:1},inplace=True)    #Brings the changes to the original dataframe

    return dataframe                                                                                #returns dataframe with modified columns with numeric data



  def OneHotEncoder(self,dataframe):                                                               #OneHotEncoder : To encode all the categorical columns with any number of categories in numeric data
    from sklearn.compose import ColumnTransformer as CT
    from sklearn.preprocessing import OneHotEncoder as ohe
    transformer = CT(
                      transformers=[('ohe',ohe(drop='first',sparse_output=False),
                      list(dataframe.select_dtypes(include='object')))]
                    )


    transformer.fit_transform(dataframe)
    sample = pd.DataFrame(transformer.fit_transform(dataframe))

    start = 0                                                                                    #There is a particular problem when sing OneHotEnoder is, it always return the numpy array with no info related to the column transformation
    renamed_columns = []                                                                         #This particular function will return the dataframe with their column header and their respective numeric values
    for i in dataframe.select_dtypes(include='object').columns:
      unique = dataframe[i].unique()
      for j in unique[1:]:
        renamed_columns.append(i+'_'+j)
    sample.columns = renamed_columns
    dataframe.drop(list(dataframe.select_dtypes(include='object')),axis=1,inplace=True)
    dataframe = pd.concat([sample,dataframe],axis=1)

    return dataframe                                                                             #returns the dataframe with One Hot Encoded values




  def Probabiltiy_Ratio(self,dataframe,target):                                                 # Probability_Ration : To encode the categorical feature based on the probability of each column value w.r.t to the target column to the
    for i in dataframe.select_dtypes(include='object').columns:
      if i!=target:
        probability = pd.DataFrame()
        probability['1'] = dataframe.groupby(i)['Attrition_Flag'].mean()
        probability['0'] = 1-probability['1']
        probability['0'].replace({0.0:0.0001},inplace=True)
        probability['P(1)/P(0)'] = probability['1']/probability['0']
        dataframe[i].replace(probability['P(1)/P(0)'].to_dict(),inplace=True)
    return dataframe                                                                            #returns dataframe with encoded values on the probability ration of P(1)/P(0)


